In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys, os
path_current = '/home/huzuntao/PycharmProjects/MPP_Powersystem/'
path_ = os.getcwd()
if path_current not in sys.path:
    sys.path.insert(1, '/home/huzuntao/PycharmProjects/MPP_Powersystem/')
elif path_ not in sys.path:
    sys.path.insert(1, path_)

from pypower_.pipsopf_solver import pipsopf_solver_gpu
import pandapower as pp

In [2]:
net = pp.create_empty_network()

# create buses
bus1 = pp.create_bus(net, vn_kv=110.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)
bus5 = pp.create_bus(net, vn_kv=110.)
bus6 = pp.create_bus(net, vn_kv=110.)

# create 110 kV lines
pp.create_line(net, bus4, bus5, length_km=90., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=90., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus2, bus3, length_km=90., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus1, bus2, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus6, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')

# create loads
p_load_1 = 10
p_load_2 = 30
pp.create_load(net, bus2, p_mw=p_load_1, controllable=False)
pp.create_load(net, bus4, p_mw=p_load_2/2, controllable=False)
pp.create_load(net, bus5, p_mw=p_load_2/2, controllable=False)
pp.create_load(net, bus6, p_mw=p_load_2/2, controllable=False)
# create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=0, max_p_mw=1000, vm_pu=1.05)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80, vm_pu=1.00, controllable=True)

costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=20)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

net.bus["min_vm_pu"] = 0.96
net.bus["max_vm_pu"] = 1.04
net.line["max_loading_percent"] = 100

In [3]:
om, ppopt, raw = pp.runopp(net, delta=1e-16, RETURN_RAW_DER=1)

In [10]:
from pypower_pych_gpu.pipsopf_solver import pipsopf_solver_torch
pipsopf_solver_torch(om, ppopt)

A_: None; eyex shape: torch.Size([16, 16]); nx : 16


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument tensors in method wrapper_CUDA_cat)

In [5]:
from concurrent.futures import ProcessPoolExecutor
from pypower__.pipsopf_solver import pipsopf_solver
def solve_problem(args):
    om, ppopt = args
    try:
        result, success, raw = pipsopf_solver(om, ppopt)
        return result, success, raw
    except Exception as e:
        # Handle exceptions or errors that occur during the execution of pipsopf_solver_gpu
        return None, False, str(e)

# Create a list of argument tuples
arg_list = [(om, ppopt), (om, ppopt), (om, ppopt), (om, ppopt)]*16*4

# Execute the function in parallel
with ProcessPoolExecutor(max_workers=8) as executor:
    results = executor.map(solve_problem, arg_list)

# Retrieve the results
for i, result in enumerate(results):
    if result[1]:
        print(f"Array {i}: {result[0]}")
    else:
        print(f"Array {i}: Failed with error: {result[2]}")



Array 0: {'baseMVA': 1, 'version': 2, 'bus': array([[ 0.00000000e+00,  3.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  1.05000000e+00,  0.00000000e+00,
         1.10000000e+02,  1.00000000e+00,  1.05000000e+00,
         1.05000000e+00,  1.03963712e+01,  4.16895132e-24,
         0.00000000e+00,  1.78695921e+02],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  1.04000000e+00,  2.60961845e-01,
         1.10000000e+02,  1.00000000e+00,  1.04000000e+00,
         9.60000000e-01,  1.02049587e+01, -3.13718981e-01,
         2.21414908e+02,  0.00000000e+00],
       [ 2.00000000e+00,  2.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  1.03334618e+00,  2.44970022e+00,
         1.10000000e+02,  1.00000000e+00,  1.04000000e+00,
         9.60000000e-01,  1.00000000e+01, -

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
from multiprocessing import Pool

def solve_problem(args):
    om, ppopt = args
    result, success, raw = pipsopf_solver_gpu(om, ppopt)
    return result, success, raw

# Create a list of argument tuples
arg_list = [(om, ppopt),  (om, ppopt),(om, ppopt),(om, ppopt)]

# Create a multiprocessing pool
with Pool(processes=8) as pool:
    # Execute the function in parallel
    results = pool.map(solve_problem, arg_list)

# Retrieve the results
for i, result in enumerate(results):
    print(f"Array {i}: {result[1]}")


CUDARuntimeError: cudaErrorInitializationError: initialization error

## pipscopf.py

In [15]:
import numpy
from cupy import ones, zeros, Inf, pi, exp, conj, r_, asarray, flatnonzero as find

from pypower_.csr_real_imag import csr_imag
from pypower_.idx_brch import F_BUS, T_BUS, RATE_A, PF, QF, PT, QT, MU_SF, MU_ST
from pypower_.idx_bus import BUS_TYPE, REF, VM, VA, MU_VMAX, MU_VMIN, LAM_P, LAM_Q
from pypower_.idx_cost import MODEL, PW_LINEAR, NCOST
from pypower_.idx_gen import GEN_BUS, PG, QG, VG, MU_PMAX, MU_PMIN, MU_QMAX, MU_QMIN
from pypower_.makeYbus import makeYbus
from pypower_.opf_consfcn import opf_consfcn
from pypower_.opf_costfcn import opf_costfcn

# from pandapower.pypower.util import sub2ind

from pypower_.opf_hessfcn import opf_hessfcn
from pypower_.pips import pips

import warnings
warnings.filterwarnings("ignore")

# pip.py
from cupy import array, Inf, any, isnan, ones, r_, finfo, \
    zeros, dot, absolute, log, arange
from cupy import flatnonzero as find
from cupy import asarray
from cupy.linalg import norm

from cupyx.scipy.sparse import vstack, hstack, eye, csr_matrix as sparse
from cupyx.scipy.sparse.linalg import spsolve

# from pandapower.pypower.pipsver import pipsver

EPS = finfo(float).eps


In [12]:
x0_init=None
out_opt=None

if out_opt is None:
    out_opt = {}

## options
verbose = ppopt['VERBOSE']
feastol = ppopt['PDIPM_FEASTOL']
gradtol = ppopt['PDIPM_GRADTOL']
comptol = ppopt['PDIPM_COMPTOL']
costtol = ppopt['PDIPM_COSTTOL']
max_it  = ppopt['PDIPM_MAX_IT']
max_red = ppopt['SCPDIPM_RED_IT']
init = ppopt['INIT']
step_control = (ppopt['OPF_ALG'] == 565)  ## OPF_ALG == 565, PIPS-sc
if feastol == 0:
    feastol = ppopt['OPF_VIOLATION']
opt = {  'feastol': feastol,
         'gradtol': gradtol,
         'comptol': comptol,
         'costtol': costtol,
         'max_it': max_it,
         'max_red': max_red,
         'step_control': step_control,
         'cost_mult': 1,
         'verbose': verbose  }

## unpack data
ppc = om.get_ppc()
baseMVA, bus, gen, branch, gencost = \
    asarray(ppc["baseMVA"]), asarray(ppc["bus"]), asarray(ppc["gen"]), \
                        asarray(ppc["branch"]), asarray(ppc["gencost"])
vv, _, nn, _ = om.get_idx()
# vv, nn = asarray(vv), asarray(nn)
## problem dimensions
nb = bus.shape[0]          ## int: number of buses
nl = branch.shape[0]       ## int: number of branches
ny = om.getN('var', 'y')   ## int: number of piece-wise linear costs

## linear constraints
A, l, u = om.linear_constraints()
if isinstance(A, numpy.ndarray):
    A = asarray(A)
if isinstance(l, numpy.ndarray):
    l = asarray(l)
if isinstance(u, numpy.ndarray):
    u = asarray(u)


## bounds on optimization vars
x0, xmin, xmax = om.getv()
x0, xmin, xmax = asarray(x0), asarray(xmin), asarray(xmax)

## build admittance matrices
Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)
if 'cupy' not in str(type(Ybus)):
    raise ValueError('Ybus must be a cupy array')
if 'cupy' not in str(type(Yf)):
    raise ValueError('Yf must be a cupy array')
if 'cupy' not in str(type(Yt)):
    raise ValueError('Yt must be a cupy array')

## try to select an interior initial point if init is not available from a previous powerflow
if init != "pf":
    ll, uu = xmin.copy(), xmax.copy()
    ll[xmin == -Inf] = -1e10   ## replace Inf with numerical proxies
    uu[xmax ==  Inf] =  1e10
    x0 = (ll + uu) / 2
    Varefs = bus[bus[:, BUS_TYPE] == REF, VA] * (pi / 180)
    ## angles set to first reference angle
    x0[vv["i1"]["Va"]:vv["iN"]["Va"]] = Varefs[0]
    if ny > 0:
#         ipwl = find(gencost[:, MODEL] == PW_LINEAR)
# #         PQ = r_[gen[:, PMAX], gen[:, QMAX]]
# #         c = totcost(gencost[ipwl, :], PQ[ipwl])
#         c = gencost.flatten('F')[sub2ind(gencost.shape, ipwl, NCOST+2*gencost[ipwl, NCOST])]    ## largest y-value in CCV data
#         x0[vv["i1"]["y"]:vv["iN"]["y"]] = max(c) + 0.1 * abs(max(c))
# #        x0[vv["i1"]["y"]:vv["iN"]["y"]] = c + 0.1 * abs(c)
        pass

if x0_init is not None:
    x0 = x0_init

## find branches with flow limits
il = find((branch[:, RATE_A] != 0) & (branch[:, RATE_A] < 1e10))
nl2 = len(il)           ## number of constrained lines


cp = om.get_cost_params()
N, Cw, H, dd, rh, kk, mm = \
    cp["N"], cp["Cw"], cp["H"], cp["dd"], cp["rh"], cp["kk"], cp["mm"]
cp = (N, Cw, H, dd, rh, kk, mm)
pack_para = (ppc, baseMVA, bus, gen, branch, gencost, il, vv, nn, ny, cp)

##-----  run opf  -----
f_fcn = lambda x, return_hessian=False: opf_costfcn(x, pack_para, return_hessian)
gh_fcn = lambda x: opf_consfcn(x, pack_para, Ybus, Yf[il, :], Yt[il,:], ppopt, il)
hess_fcn = lambda x, lmbda, cost_mult: opf_hessfcn(x, lmbda, pack_para, Ybus, Yf[il, :], Yt[il, :], ppopt, il, cost_mult)


In [13]:
solution = pips(f_fcn, x0, A, l, u, xmin, xmax, gh_fcn, hess_fcn, opt)
x, f, info, lmbda, output = solution["x"], solution["f"], \
        solution["eflag"], solution["lmbda"], solution["output"]

success = (info > 0)

A_: None; eyex shape: (16, 16); nx : 16
max it: 150; i: 1
max it: 150; i: 2
max it: 150; i: 3
max it: 150; i: 4
max it: 150; i: 5
max it: 150; i: 6
max it: 150; i: 7
max it: 150; i: 8
max it: 150; i: 9
max it: 150; i: 10
max it: 150; i: 11
max it: 150; i: 12
max it: 150; i: 13
max it: 150; i: 14
max it: 150; i: 15
max it: 150; i: 16
max it: 150; i: 17
max it: 150; i: 18
max it: 150; i: 19
max it: 150; i: 20
max it: 150; i: 21
max it: 150; i: 22
max it: 150; i: 23
max it: 150; i: 24
max it: 150; i: 25
max it: 150; i: 26


## pip.py

In [63]:
if isinstance(f_fcn, dict):  ## problem dict
    p = f_fcn
    f_fcn = p['f_fcn']
    x0 = p['x0']
    if 'opt' in p: opt = p['opt']
    if 'hess_fcn' in p: hess_fcn = p['hess_fcn']
    if 'gh_fcn' in p: gh_fcn = p['gh_fcn']
    if 'xmax' in p: xmax = p['xmax']
    if 'xmin' in p: xmin = p['xmin']
    if 'u' in p: u = p['u']
    if 'l' in p: l = p['l']
    if 'A' in p: A = p['A']
# print(f"isinstance(f_fcn, dict): {isinstance(f_fcn, dict)}; x0 shape: {x0.shape}")

In [64]:

nx = x0.shape[0]                       # int: number of variables
nA = A.shape[0] if A is not None else 0 # int: number of original linear constr

# default argument values
if l is None or len(l) == 0: l = -Inf * ones(nA)
if u is None or len(u) == 0: u =  Inf * ones(nA)
if xmin is None or len(xmin) == 0: xmin = -Inf * ones(x0.shape[0])
if xmax is None or len(xmax) == 0: xmax =  Inf * ones(x0.shape[0])
if gh_fcn is None:
    nonlinear = False
    gn = array([])
    hn = array([])
else:
    nonlinear = True

if opt is None: opt = {}
# options
if "feastol" not in opt:
    opt["feastol"] = 1e-06
if "gradtol" not in opt:
    opt["gradtol"] = 1e-06
if "comptol" not in opt:
    opt["comptol"] = 1e-06
if "costtol" not in opt:
    opt["costtol"] = 1e-06
if "max_it" not in opt:
    opt["max_it"] = 150
if "max_red" not in opt:
    opt["max_red"] = 20
if "step_control" not in opt:
    opt["step_control"] = False
if "cost_mult" not in opt:
    opt["cost_mult"] = 1
if "verbose" not in opt:
    opt["verbose"] = 0

# initialize history
hist = []

# constants
xi = 0.99995
sigma = 0.1
z0 = 1
alpha_min = 1e-8
rho_min = 0.95
rho_max = 1.05
mu_threshold = 1e-5

# initialize
i = 0                       # iteration counter
converged = False           # flag
eflag = False               # exit flag

In [99]:

# add var limits to linear constraints
eyex = eye(nx, nx, format="csr")
# print(f"A_: {A}; eyex shape: {eyex.shape}; nx : {nx}")
AA = eyex if A is None else vstack([eyex, A], "csr")
ll = r_[xmin, l]
uu = r_[xmax, u]

In [114]:

# split up linear constraints
ieq = find( absolute(uu - ll) <= EPS )
igt = find( (uu >=  1e10) & (ll > -1e10) )
ilt = find( (ll <= -1e10) & (uu <  1e10) )
ibx = find( (absolute(uu - ll) > EPS) & (uu < 1e10) & (ll > -1e10) )
# zero-sized sparse matrices unsupported
# print(f'ieq_: {ieq};AA shape: {AA.shape}')
Ae = AA[ieq, :] if len(ieq) else None
if len(ilt) or len(igt) or len(ibx):
    idxs = [(1, ilt), (-1, igt), (1, ibx), (-1, ibx)]
    Ai = vstack([sig * AA[idx, :] for sig, idx in idxs if len(idx)], 'csr')
else:
    Ai = None
be = uu[ieq]
bi = r_[uu[ilt], -ll[igt], uu[ibx], -ll[ibx]]

In [169]:

# TODO: the following function requrires be in cupy
# evaluate cost f(x0) and constraints g(x0), h(x0)
x = x0
f, df = f_fcn(x)                 # cost
f = f * opt["cost_mult"]
df = df * opt["cost_mult"]
if 1:
    hn, gn, dhn, dgn = gh_fcn(x)        # nonlinear constraints
    # h = hn if Ai is None else r_[hn.reshape(len(hn),), Ai * x - bi] # inequality constraints
    # g = gn if Ae is None else r_[gn, Ae * x - be] # equality constraints
    #
    # if (dhn is None) and (Ai is None):
    #     dh = None
    # elif dhn is None:
    #     dh = Ai.T
    # elif Ai is None:
    #     dh = dhn
    # else:
    #     dh = hstack([dhn, Ai.T])
    #
    # if (dgn is None) and (Ae is None):
    #     dg = None
    # elif dgn is None:
    #     dg = Ae.T
    # elif Ae is None:
    #     dg = dgn
    # else:
    #     dg = hstack([dgn, Ae.T])
# else:
#     h = -bi if Ai is None else Ai * x - bi        # inequality constraints
#     g = -be if Ae is None else Ae * x - be        # equality constraints
#     dh = None if Ai is None else Ai.T     # 1st derivative of inequalities
#     dg = None if Ae is None else Ae.T     # 1st derivative of equalities

In [201]:
# f, df = f_fcn(x)
from cupy import zeros, ones, conj, exp, Inf, arange, r_
# from cupy import concatenate as r_ # concatenate is similar with np.r_, but the specifics will depend on your use case

# from scipy.sparse import lil_matrix, vstack, hstack, csr_matrix as sparse
from cupyx.scipy.sparse import vstack, hstack, csr_matrix as sparse

from pypower_.idx_gen import GEN_BUS, PG, QG
from pypower_.idx_brch import F_BUS, T_BUS, RATE_A

# 所有的函数都需要转换成Cupy的
from pypower_.makeSbus import makeSbus
from pypower_.dSbus_dV import dSbus_dV
from pypower_.dIbr_dV import dIbr_dV
from pypower_.dSbr_dV import dSbr_dV
from pypower_.dAbr_dV import dAbr_dV

from pypower_.csr_real_imag import csr_real, csr_imag, cr_real, cr_imag


## unpack data

baseMVA, bus, gen, branch = pack_para[1], pack_para[2], pack_para[3], pack_para[4]
vv = pack_para[7]

## problem dimensions
nb = bus.shape[0]          ## int: number of buses
nl = branch.shape[0]       ## int: number of branches
ng = gen.shape[0]          ## int: number of dispatchable injections
nxyz = len(x)              ## int: total number of control vars of all types

## set default constrained lines
if il is None:
    il = arange(nl)         ## all lines have limits by default
nl2 = len(il)              ## int: number of constrained lines
#
# grab Pg & Qg
Pg = x[vv["i1"]["Pg"]:vv["iN"]["Pg"]]  ## active generation in p.u.
Qg = x[vv["i1"]["Qg"]:vv["iN"]["Qg"]]  ## reactive generation in p.u.

## put Pg & Qg back in gen
gen[:, PG] = Pg * baseMVA  ## active generation in MW
gen[:, QG] = Qg * baseMVA  ## reactive generation in MVAr

## rebuild Sbus
Sbus = makeSbus(baseMVA, bus, gen) ## net injected power in p.u.

In [211]:

## ----- evaluate constraints -----
## reconstruct V
Va = x[vv["i1"]["Va"]:vv["iN"]["Va"]]
Vm = x[vv["i1"]["Vm"]:vv["iN"]["Vm"]]
V = Vm * exp(1j * Va)

## evaluate power flow equations
mis = V * (Ybus * V).conj() - Sbus
# print(f"V: {V}; Ybus: {Ybus}; Ybus*V: {Ybus * V}")

##----- evaluate constraint function values -----
## first, the equality constraints (power flow)
g = r_[ mis.real,            ## active power mismatch for all buses
        mis.imag ]           ## reactive power mismatch for all buses

## then, the inequality constraints (branch flow limits)
if nl2 > 0:
    flow_max = (branch[il, RATE_A] / baseMVA)**2
    flow_max[flow_max == 0] = Inf
    if ppopt['OPF_FLOW_LIM'] == 2:       ## current magnitude limit, |I|
        If = Yf * V
        It = Yt * V
        h =  r_[ If * If.conj() - flow_max,     ## branch I limits (from bus)
                It * It.conj() - flow_max ].real   ## branch I limits (to bus)
    else:
        ## compute branch power flows
        ## complex power injected at "from" bus (p.u.)
        Sf = V[ branch[il, F_BUS].astype(int) ] * (Yf * V).conj()
        ## complex power injected at "to" bus (p.u.)
        St = V[ branch[il, T_BUS].astype(int) ] * (Yt * V).conj()
        if ppopt['OPF_FLOW_LIM'] == 1:   ## active power limit, P (Pan Wei)
            h = r_[ csr_real(Sf)**2 - flow_max,   ## branch P limits (from bus)
                    csr_real(St)**2 - flow_max ]  ## branch P limits (to bus)
        else:                ## apparent power limit, |S|
            h = cr_real(
                r_[ Sf * Sf.conj() - flow_max, ## branch S limits (from bus)
                    St * St.conj() - flow_max ])  ## branch S limits (to bus)
else:
    h = zeros((0, 1))

In [223]:

##----- evaluate partials of constraints -----
## index ranges
iVa = arange(vv["i1"]["Va"], vv["iN"]["Va"])
iVm = arange(vv["i1"]["Vm"], vv["iN"]["Vm"])
iPg = arange(vv["i1"]["Pg"], vv["iN"]["Pg"])
iQg = arange(vv["i1"]["Qg"], vv["iN"]["Qg"])
iVaVmPgQg = r_[iVa, iVm, iPg, iQg].T

## compute partials of injected bus powers
dSbus_dVm, dSbus_dVa = dSbus_dV(Ybus, V)           ## w.r.t. V
## Pbus w.r.t. Pg, Qbus w.r.t. Qg
neg_Cg = sparse((-ones(ng), (gen[:, GEN_BUS], arange(ng))), (nb, ng))

## construct Jacobian of equality constraints (power flow) and transpose it
dg = sparse(zeros((2 * nb, nxyz)))
blank = sparse((nb, ng))
dg[:, iVaVmPgQg] = vstack([
        ## P mismatch w.r.t Va, Vm, Pg, Qg
        hstack([csr_real(dSbus_dVa), csr_real(dSbus_dVm), neg_Cg, blank]),
        ## Q mismatch w.r.t Va, Vm, Pg, Qg
        hstack([cr_imag(dSbus_dVa), cr_imag(dSbus_dVm), blank, neg_Cg])
    ], "csr")
dg = dg.T

if nl2 > 0:
    ## compute partials of Flows w.r.t. V
    if ppopt['OPF_FLOW_LIM'] == 2:     ## current
        dFf_dVa, dFf_dVm, dFt_dVa, dFt_dVm, Ff, Ft = \
                dIbr_dV(branch[il, :], Yf, Yt, V)
    else:                  ## power
        dFf_dVa, dFf_dVm, dFt_dVa, dFt_dVm, Ff, Ft = \
                dSbr_dV(branch[il, :], Yf, Yt, V)
    if ppopt['OPF_FLOW_LIM'] == 1:     ## real part of flow (active power)
        dFf_dVa = csr_real(dFf_dVa)
        dFf_dVm = csr_real(dFf_dVm)
        dFt_dVa = csr_real(dFt_dVa)
        dFt_dVm = csr_real(dFt_dVm)
        Ff = csr_real(Ff)
        Ft = csr_real(Ft)

    ## squared magnitude of flow (of complex power or current, or real power)
    df_dVa, df_dVm, dt_dVa, dt_dVm = \
            dAbr_dV(dFf_dVa, dFf_dVm, dFt_dVa, dFt_dVm, Ff, Ft)

    ## construct Jacobian of inequality constraints (branch limits)
    ## and transpose it.
    dh = sparse(zeros((2 * nl2, nxyz)))
    dh[:, r_[iVa, iVm].T] = vstack([
            hstack([df_dVa, df_dVm]),    ## "from" flow limit
            hstack([dt_dVa, dt_dVm])     ## "to" flow limit
        ], "csr")
    dh = dh.T
else:
    dh = None


In [143]:

# some dimensions
neq = g.shape[0]           # int: number of equality constraints
niq = h.shape[0]           # int: number of inequality constraints
neqnln = gn.shape[0]       # int: number of nonlinear equality constraints
niqnln = hn.shape[0]       # int: number of nonlinear inequality constraints
nlt = len(ilt)             # int: number of upper bounded linear inequalities
ngt = len(igt)             # int: number of lower bounded linear inequalities
nbx = len(ibx)             # int: number of doubly bounded linear inequalities

# initialize gamma, lam, mu, z, e
gamma = 1                  # barrier coefficient
lam = zeros(neq)
z = z0 * ones(niq)
mu = z0 * ones(niq)
k = find(h < -z0)
z[k] = -h[k]
k = find((gamma / z) > z0)
mu[k] = gamma / z[k]
e = ones(niq)

# check tolerance
f0 = f
if opt["step_control"]:
    L = f + dot(lam, g) + dot(mu, h + z) - gamma * sum(log(z))

Lx = df.copy()
Lx = Lx + dg * lam if dg is not None else Lx
Lx = Lx + dh * mu  if dh is not None else Lx

maxh = zeros(1) if len(h) == 0 else max(h)

gnorm = norm(g, Inf) if len(g) else 0.0
lam_norm = norm(lam, Inf) if len(lam) else 0.0
mu_norm = norm(mu, Inf) if len(mu) else 0.0
znorm = norm(z, Inf) if len(z) else 0.0
feascond = \
    max([gnorm, maxh]) / (1 + max([norm(x, Inf), znorm]))
gradcond = \
    norm(Lx, Inf) / (1 + max([lam_norm, mu_norm]))
compcond = dot(z, mu) / (1 + norm(x, Inf))
costcond = absolute(f - f0) / (1 + absolute(f0))

# save history
hist.append({'feascond': feascond, 'gradcond': gradcond,
    'compcond': compcond, 'costcond': costcond, 'gamma': gamma,
    'stepsize': 0, 'obj': f / opt["cost_mult"], 'alphap': 0, 'alphad': 0})

if opt["verbose"]: # pragma: no cover
    s = '-sc' if opt["step_control"] else ''
    v = {'Name': 'PIPS',
           'Version': '1.0',
           'Release':  '',
           'Date': '07-Feb-2011'}
    print('Python Interior Point Solver - PIPS%s, Version %s, %s' %
                (s, v['Version'], v['Date']))
    if opt['verbose'] > 1:
        print(" it    objective   step size   feascond     gradcond     "
              "compcond     costcond  ")
        print("----  ------------ --------- ------------ ------------ "
              "------------ ------------")
        print("%3d  %12.8g %10s %12g %12g %12g %12g" %
            (i, (f / opt["cost_mult"]), "",
             feascond, gradcond, compcond, costcond))

if feascond < opt["feastol"] and gradcond < opt["gradtol"] and \
    compcond < opt["comptol"] and costcond < opt["costtol"]:
    converged = True
    if opt["verbose"]:
        print("Converged!")


### Iteration loop

In [58]:
# do Newton iterations
i=0
converged = False
# x = x0
while (not converged) and (i < 10):
    # update iteration counter
    i += 1
    print(f"max it: {opt['max_it']}; i: {i}")

    # compute update step
    lmbda = {"eqnonlin": lam[range(neqnln)],
             "ineqnonlin": mu[range(niqnln)]}
    if nonlinear:
        if hess_fcn is None:
            print("pips: Hessian evaluation via finite differences "
                  "not yet implemented.\nPlease provide "
                  "your own hessian evaluation function.")
        Lxx = hess_fcn(x, lmbda, opt["cost_mult"])
    else:
        _, _, d2f = f_fcn(x, True)      # cost
        Lxx = d2f * opt["cost_mult"]
    rz = arange(len(z))
    zinvdiag = sparse((1.0 / z, (rz, rz))) if len(z) else None
    rmu = arange(len(mu))
    mudiag = sparse((mu, (rmu, rmu))) if len(mu) else None
    dh_zinv = None if dh is None else dh * zinvdiag
    M = Lxx if dh is None else Lxx + dh_zinv * mudiag * dh.T
    N = Lx if dh is None else Lx + dh_zinv * (mudiag * h + gamma * e)

    Ab = sparse(M) if dg is None else vstack([
        hstack([M, dg]),
        hstack([dg.T, sparse((neq, neq))])
    ])
    bb = r_[-N, -g]

    dxdlam = spsolve(Ab.tocsr(), bb)

    if any(isnan(dxdlam)):
        if opt["verbose"]:
            print('\nNumerically Failed\n')
        eflag = -1
        break

    dx = dxdlam[:nx]
    dlam = dxdlam[nx:nx + neq]
    dz = -h - z if dh is None else -h - z - dh.T * dx
    dmu = -mu if dh is None else -mu + zinvdiag * (gamma * e - mudiag * dz)

    # do the update
    k = find(dz < 0.0)
    alphap = min([xi * min(z[k] / -dz[k]), 1]) if len(k) else 1.0
    k = find(dmu < 0.0)
    alphad = min([xi * min(mu[k] / -dmu[k]), 1]) if len(k) else 1.0
    x = x + alphap * dx
    z = z + alphap * dz
    lam = lam + alphad * dlam
    mu = mu + alphad * dmu
    if niq > 0:
        gamma = sigma * dot(z, mu) / niq

    # evaluate cost, constraints, derivatives
    f, df = f_fcn(x)             # cost
    f = f * opt["cost_mult"]
    df = df * opt["cost_mult"]
    if nonlinear:
        hn, gn, dhn, dgn = gh_fcn(x)                   # nln constraints
#            g = gn if Ai is None else r_[gn, Ai * x - bi] # ieq constraints
#            h = hn if Ae is None else r_[hn, Ae * x - be] # eq constraints
        h = hn if Ai is None else r_[hn.reshape(len(hn),), Ai * x - bi] # ieq constr
        g = gn if Ae is None else r_[gn, Ae * x - be]  # eq constr

        if (dhn is None) and (Ai is None):
            dh = None
        elif dhn is None:
            dh = Ai.T
        elif Ai is None:
            dh = dhn
        else:
            dh = hstack([dhn, Ai.T])

        if (dgn is None) and (Ae is None):
            dg = None
        elif dgn is None:
            dg = Ae.T
        elif Ae is None:
            dg = dgn
        else:
            dg = hstack([dgn, Ae.T])
    else:
        h = -bi if Ai is None else Ai * x - bi    # inequality constraints
        g = -be if Ae is None else Ae * x - be    # equality constraints
        # 1st derivatives are constant, still dh = Ai.T, dg = Ae.T

    Lx = df
    Lx = Lx + dg * lam if dg is not None else Lx
    Lx = Lx + dh * mu  if dh is not None else Lx

    if len(h) == 0:
        maxh = zeros(1)
    else:
        maxh = max(h)

    gnorm = norm(g, Inf) if len(g) else 0.0
    lam_norm = norm(lam, Inf) if len(lam) else 0.0
    mu_norm = norm(mu, Inf) if len(mu) else 0.0
    znorm = norm(z, Inf) if len(z) else 0.0
    feascond = \
        max([gnorm, maxh]) / (1 + max([norm(x, Inf), znorm]))
    gradcond = \
        norm(Lx, Inf) / (1 + max([lam_norm, mu_norm]))
    compcond = dot(z, mu) / (1 + norm(x, Inf))
    costcond = float(absolute(f - f0) / (1 + absolute(f0)))
    hist.append({'feascond': feascond, 'gradcond': gradcond,
        'compcond': compcond, 'costcond': costcond, 'gamma': gamma,
        'stepsize': norm(dx), 'obj': f / opt["cost_mult"],
        'alphap': alphap, 'alphad': alphad})

    if opt["verbose"] > 1:
        print("%3d  %12.8g %10.5g %12g %12g %12g %12g" %
            (i, (f / opt["cost_mult"]), norm(dx), feascond, gradcond,
             compcond, costcond))

    if feascond < opt["feastol"] and gradcond < opt["gradtol"] and \
        compcond < opt["comptol"] and costcond < opt["costtol"]:
        converged = True
        if opt["verbose"]:
            print("Converged!")
    else:
        if any(isnan(x)) or (alphap < alpha_min) or \
            (alphad < alpha_min) or (gamma < EPS) or (gamma > 1.0 / EPS):
            if opt["verbose"]:
                print("Numerically failed.")
            eflag = -1
            break
        f0 = f

        if opt["step_control"]:
            L = f + dot(lam, g) + dot(mu, (h + z)) - gamma * sum(log(z))


max it: 150; i: 1
max it: 150; i: 2
max it: 150; i: 3
max it: 150; i: 4
max it: 150; i: 5
max it: 150; i: 6
max it: 150; i: 7
max it: 150; i: 8
max it: 150; i: 9
max it: 150; i: 10


In [46]:
x0

array([-4.30104384e-23, -4.30104384e-23, -4.30104384e-23, -4.30104384e-23,
       -4.30104384e-23, -4.30104384e-23,  1.05000000e+00,  1.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        5.00000000e+02,  4.00000000e+01,  0.00000000e+00,  0.00000000e+00])

In [47]:
x

array([-8.21484045e-17, -6.57891068e-02, -1.41378963e-01, -2.32842159e-01,
       -2.77433368e-01, -1.75956237e-01,  1.05000001e+00,  1.04209111e+00,
        9.95034874e-01,  9.65145582e-01,  9.48151478e-01,  9.81283823e-01,
        2.83777257e+01,  2.03511287e+01, -1.44583734e+01, -2.46173381e+01])

In [49]:
x_0, _, _ = om.getv()
x_0

array([ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.05,  1.  ,  1.  ,
        1.  ,  1.  ,  1.  ,  0.  , 80.  ,  0.  ,  0.  ])